In [45]:
from imutils import contours
import numpy as np
import argparse
import cv2
import pytesseract
import import_ipynb
import myutils
import easyocr

In [46]:
def cv_show(name,img):
    cv2.imshow(name,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows();

In [47]:
img = cv2.imread('plate_11.jpeg')

In [48]:
ratio = img.shape[0]/500.0
orig = img.copy();
img = myutils.resize(orig,height=500)


In [49]:
cv_show('img',img)
img.shape

(500, 666, 3)

In [50]:

img=cv2.GaussianBlur(img,(5,5),0)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
rectKern = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 10))
blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, rectKern)
cv_show("Tophat", blackhat)

In [51]:
squareKern = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
light = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, squareKern)
light = cv2.threshold(light, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
cv_show("Light Regions", light)

In [52]:
gradX = cv2.Sobel(blackhat, ddepth=cv2.CV_32F,dx=1, dy=0, ksize=-1)
gradX = np.absolute(gradX)
(minVal, maxVal) = (np.min(gradX), np.max(gradX))
gradX = 255 * ((gradX - minVal) / (maxVal - minVal))
gradX = gradX.astype("uint8")
cv_show("Scharr", gradX)

In [53]:
gradX = cv2.GaussianBlur(gradX, (5, 5), 0)
gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKern)
thresh = cv2.threshold(gradX, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
cv_show("Grad Thresh", thresh)

In [54]:
thresh = cv2.erode(thresh, None, iterations=2)
thresh = cv2.dilate(thresh, None, iterations=2)

In [55]:
thresh = cv2.bitwise_and(thresh, thresh, mask=light)
thresh = cv2.dilate(thresh, None, iterations=2)
thresh = cv2.erode(thresh, None, iterations=1)
cv_show("Final", thresh)

In [56]:
cnts,hierarchy = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
draw_img = img.copy();
cv2.drawContours(draw_img,cnts,-1,(0,0,255),3)
cv_show('cnts',draw_img)

In [57]:
lpCnt = None
roi = None
		# loop over the license plate candidate contours
for c in cnts:
    # compute the bounding box of the contour and then use
    # the bounding box to derive the aspect ratio
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w / float(h)
    print(ar)
    roi = img[y:y+h,x:x+w]
    #cv_show('roi',roi);
    
    if ar >= 3 and ar <= 6:
        # store the license plate contour and extract the
        # license plate from the grayscale image and then
        # threshold it
        lpCnt = c
        img_seg = img[y:y+h,x:x+w]
        licensePlate = img[y:y + h, x:x + w]

        break

0.6666666666666666
0.4782608695652174
0.5
0.7272727272727273
0.42857142857142855
0.75
0.09302325581395349
5.3076923076923075


In [58]:
# licensePlate = myutils.enlarge_img(licensePlate,150)
licensePlate = cv2.GaussianBlur(licensePlate,(5,5),0)
licensePlate  = cv2.cvtColor(licensePlate,cv2.COLOR_BGR2GRAY)
licensePlate = myutils.resize(licensePlate,height=60)
cv_show('tmp',licensePlate)
licensePlate.shape

(60, 318)

In [59]:
licensePlate = cv2.threshold(licensePlate, 0, 255,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]


In [60]:
lp_close = cv2.morphologyEx(licensePlate,cv2.MORPH_CLOSE,squareKern)
lp_close = cv2.morphologyEx(licensePlate,cv2.MORPH_OPEN,squareKern)
lp_close = cv2.erode(lp_close,squareKern,iterations=1)

In [61]:
cv_show('lp_close',lp_close)

In [62]:
result = np.zeros(licensePlate.shape, dtype=np.uint8)
cv_show('res',result)

In [63]:
lp_close = licensePlate.copy()
contours,hierarchy = cv2.findContours(lp_close,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE);
# cv2.drawContours(img_seg,contours,-1,(0,0,255),2)
# cv_show('img_seg',img_seg)

In [64]:
contours = myutils.sort_contours(contours)[0]
for c in contours:
    x,y,w,h = cv2.boundingRect(c);
    roi = lp_close[y:y+h,x:x+w];
    cnt_white = np.sum(roi == 255);
    cv_show('roi',roi) 
    rat = w/float(h)
    print(cnt_white)
    print("rat = ",rat)
    if (rat >= 0.5 and rat <= 1.2) or (cnt_white >= 200 and cnt_white <= 600) :
        result[y:y+h,x:x+w] = lp_close[y:y+h,x:x+w];

776
rat =  0.5185185185185185
873
rat =  0.5370370370370371
98
rat =  2.2857142857142856
782
rat =  0.5178571428571429
431
rat =  0.20754716981132076
841
rat =  0.5849056603773585
65
rat =  1.5714285714285714
712
rat =  0.5178571428571429
899
rat =  0.5185185185185185
132
rat =  0.06666666666666667


In [65]:
# result = cv2.dilate(result,None,iterations=1);
invert = 255-result
cv_show('invert',invert);

In [66]:
val = pytesseract.image_to_string(invert, config = '--psm 7')
print(val)

Sk 314 CK

